In [45]:
import sympy as sp
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Math, Latex, Markdown

InteractiveShell.ast_node_interactivity = "all"

## Поиск собственных значений и векторов

In [ ]:
class Eigenvector:
    def __init__(self, eigenvalue, alpha, eigenspace):
        self.eigenvalue = eigenvalue
        self.alpha = alpha
        self.eigenspace = eigenspace
        self.gamma = len(eigenspace)


def find_eigenvectors(M: sp.Matrix, name: str = "M") -> list:
    display(Markdown("## Посчитаем собственные вектора матрицы"))

    display(Markdown("### Сначала найдем характеристический многочлен"))
    lamb = sp.Symbol(r"\lambda")
    M_char_matrix = sp.Determinant(M - lamb * sp.eye(M.shape[0]))
    display(Math(name + " - " + lamb.name + r" \cdot E = "))
    display(M_char_matrix)
    M_char = M_char_matrix.simplify()
    display(M_char)

    display(Markdown("### Найдем его корни и их алгебраические кратности"))

    M_eigenvectors = M.eigenvects()

    for i in range(len(M_eigenvectors)):
        eigenvalue, alpha, _ = M_eigenvectors[i]
        display(Math(
            lamb.name + "_" + str(i + 1) + " = " + str(eigenvalue) + ", " + r"\alpha(" + str(eigenvalue) + ") = " + str(
                alpha)))

    display(Markdown("### Для каждого собственного значения найдем собственное подпространство"))

    M_result_eigenvectors = []
    for i in range(len(M_eigenvectors)):
        eigenvalue, alpha, eigenspace = M_eigenvectors[i]
        display(Math(lamb.name + "_" + str(i + 1) + " = " + str(eigenvalue) + ":"))

        M_eigen_system = M - eigenvalue * sp.eye(M.shape[0])
        display(M_eigen_system)

        display(Markdown("Перейдем к улучшенному ступенчатому виду"))
        M_eigen_system_rref, _ = M_eigen_system.rref()
        display(M_eigen_system_rref)

        display(Markdown("Т.о. собственное подпространство состоит из векторов: "))
        for eigenvector in eigenspace:
            display(eigenvector)

        gamma = len(eigenspace)
        display(Math(r"\gamma(" + str(eigenvalue) + ") = " + str(gamma)))

        M_result_eigenvectors.append(Eigenvector(eigenvalue, alpha, eigenspace))

    return M_result_eigenvectors


find_eigenvectors(sp.Matrix([
    [2, -1, -2],
    [-1, 5, 1],
    [-2, 1, 2],
]), "Q")

## Диагонализация

In [ ]:
def diagonalize(M: sp.Matrix, name: str = "M") -> sp.Matrix:
    M_eigenvectors = find_eigenvectors(M, name)

    is_diagonalizable = True
    for eigenvector in M_eigenvectors:
        is_diagonalizable = is_diagonalizable and (eigenvector.alpha == eigenvector.gamma)

    if is_diagonalizable:
        display()
    else:
        display()